In [1]:
import pandas as pd
test = [ [1,2,3], [4,5,6], [7,8,9]]

test_df = pd.DataFrame(test, columns=['a', 'b', 'c'])
print(test_df)
print(test_df.iloc[:]['a'])

   a  b  c
0  1  2  3
1  4  5  6
2  7  8  9
0    1
1    4
2    7
Name: a, dtype: int64


In [2]:
from datetime import datetime
now = datetime.now()
print(now)

dt_string = now.strftime("%Y-%m-%d-%H:%M:%S")
print("date and time =", dt_string)	

2021-10-30 10:51:32.438395
date and time = 2021-10-30-10:51:32


In [17]:
import re
def preprocess(text):
    text = re.sub(r"\n", " ", text)
    text = re.sub(r"\\n", " ", text)
    #text = re.sub(r"\'", "", text) -> 이거 풀면 성능 하락
    text = re.sub(r"\s+", " ", text)
    text = re.sub(r"#", " ", text)
    text = re.sub(
        r"[^a-zA-Z0-9가-힣ㄱ-ㅎㅏ-ㅣぁ-ゔァ-ヴー々〆〤一-龥<>()\s\.\?!》《≪≫\'<>〈〉:‘’%,『』「」＜＞・\"-“”∧]",
        "",
        text,
    )
    return text

In [3]:
wiki_df = pd.read_json('../../data/wikipedia_documents_del.json', orient='index')
# context_id = 3
len(wiki_df['text'].unique())

56737

In [4]:
wiki_df.head(10)

,text,corpus_source,url,domain,title,author,html,document_id
0,매우 다양한 분야의 연구에서 주어진 문제에 대하여 적절한 정보를 수집하고 분석하여 ...,위키피디아,TODO,NaN,통계학,NaN,NaN,18
1,"고대 일본에서 한자의 일부 획을 따서 문자로 삼은 것은 일찍이 7세기부터였으나, ...",위키피디아,TODO,NaN,가타카나,NaN,NaN,37
2,가난 때문에 중학교에 진학하지 못하고 아버지의 농사를 도왔다. 가난에서 벗어나려고 ...,위키피디아,TODO,NaN,정주영,NaN,NaN,40
3,그러나 고 김대중 전 대통령이 제15대 대통령에 당선되어 1998년 2월 25일 국...,위키피디아,TODO,NaN,정주영,NaN,NaN,46
4,선형대수학은 2차원 혹은 3차원의 직교 좌표계에 대한 연구로 부터 시작되었다.\n\...,위키피디아,TODO,NaN,선형대수학,NaN,NaN,62
5,학부과정에서 가르치는 선형대수학의 내용들은 다음과 같다. 다만 이 내용은 일반적으로...,위키피디아,TODO,NaN,선형대수학,NaN,NaN,63
6,아칸소주 호프에서 태어난 윌리엄 제퍼슨 클린턴은 5번째 세대의 아칸소주민이었다. 그...,위키피디아,TODO,NaN,빌 클린턴,NaN,NaN,74
7,1974년 클린턴은 자신의 10대 시절 이래 자신이 원했던 정치 경력을 시작하기로 ...,위키피디아,TODO,NaN,빌 클린턴,NaN,NaN,75
8,즉각적으로 정권을 인수받은 후 클린턴은 1993년 가족 의료 법안에 관한 대선 공약...,위키피디아,TODO,NaN,빌 클린턴,NaN,NaN,78
9,클린턴의 첫 기간의 말기가 접근하면서 새로운 스캔들이 일어났다. 스캔들은 클린턴과 ...,위키피디아,TODO,NaN,빌 클린턴,NaN,NaN,79


In [18]:
from collections import defaultdict
from tqdm.auto import tqdm
group_by_answer = defaultdict(str)

for idx, row in tqdm(wiki_df.iterrows()):
    group_by_answer[row['title']] += preprocess(row['text'])

# print(group_by_answer)

In [19]:
merge_wiki = {}

for idx, title in tqdm(enumerate(group_by_answer)):
    merge_wiki[idx] = {'text': group_by_answer[title], 'title': title, 'document_id': idx}

In [20]:
import json

with open('merge_wiki.json', 'w', encoding='utf-8') as json_file:
    json.dump(merge_wiki, json_file)

In [21]:
wiki_df = pd.read_json('merge_wiki.json', orient='index')
# context_id = 3
wiki_df.head(10)

,text,title,document_id
0,매우 다양한 분야의 연구에서 주어진 문제에 대하여 적절한 정보를 수집하고 분석하여 ...,통계학,0
1,"고대 일본에서 한자의 일부 획을 따서 문자로 삼은 것은 일찍이 7세기부터였으나, 오...",가타카나,1
2,가난 때문에 중학교에 진학하지 못하고 아버지의 농사를 도왔다. 가난에서 벗어나려고 ...,정주영,2
3,선형대수학은 2차원 혹은 3차원의 직교 좌표계에 대한 연구로 부터 시작되었다. 선형...,선형대수학,3
4,아칸소주 호프에서 태어난 윌리엄 제퍼슨 클린턴은 5번째 세대의 아칸소주민이었다. 그...,빌 클린턴,4
5,보통 오른손잡이는 반대쪽 왼손으로 넥 부분을 잡고 현을 누르며 오른손으로 현을 탄현...,기타,5
6,블랙데이가 언제부터 어떻게 시작됐는지는 정확히 알 수 없다. 처음으로 언급된 것은 ...,블랙데이,6
7,에릭 레이먼드가 쓴 〈성당과 시장〉(The Cathedral and the Baza...,성당과 시장,7
8,기시 노부스케 내각 기간인 1959년부터 1960년까지 미일상호방위조약 체결을 둘러...,자유민주당 (일본),8
9,가와바타 야스나리는 일본 문학사에 찬연히 빛나는 수많은 명작을 남긴 근현대 일본 문...,가와바타 야스나리,9


In [22]:
wiki_df[wiki_df['title'] == '아오조라 문고']['text'][395]

'아오조라 문고(靑空文庫, あおぞらぶんこ|아오조라 분고)는 ‘일본어판 구텐베르크 프로젝트’로 불리는 일본의 인터넷 전자도서관으로, 저작권이 풀린 문학작품을 수집, 전자문서화해서 인터넷에 공개하고 있다. 저자 사후 50년이 지난 메이지, 쇼와 시대 초기의 일본 문학 작품이 그 대부분을 차지하고 있고, 일본어 외 문학 작품의 일본어 번역 작품도 다수 있다. 1997년 2월 도미타 미치오, 노구치 에이치, 야마키 미에, 란무로 사테이 등 4명이 창설하여 시작되었다. 2016년 연간 방문객수는 940만 건 이상이다. 아오조라 문고에 수록된 작품은 JIS X 0208에 해당되는 한자 범위 내에서 자원봉사자에 의해 아오조라 문고 형식 텍스트파일이나 HTML 파일로 전자화된다. 또 아오조라 문고 수록파일 취급기준에 따라 자유롭게 이용할 수 있기 때문에, 수록된 작품을 PC는 물론 PDA와 휴대전화로도 볼 수 있다. 텍스트 파일을 큰 글자로 인쇄하거나 전용 소프트웨어에 불러들여 시각장애인용으로 이용하는 방안도 기대되고 있다. 아오조라 문고의 열람 소프트웨어는 따로 개발 및 제공되고 있는 것은 없지만, 전자사전이나 아이폰용 어플리케이션 등은 타사에서 개발하여 출시되어 있다저자 사망 이후 50년이 지나 저작권이 소멸한 메이지 시대부터 쇼와 시대 초기까지의 서적 대부분이 존재한다. 외국 번역작품이나 저자가 무료보기를 인정한 현대작품도 포함된다. 장르는 정치부터 취미까지 다양하지만, 비교적 문학작품(시대소설, 추리소설등의 오락작품 포함)이 많다. 유명작가의 작품이 모두 갖춰져있진 않지만 그래도 일본어작품에 관련해서는 충실하게 갖춰진 편이다. (번역작품의 경우 번역저작권을 문제로 수가 많지 않다.) 잘 알려지지 않은 작품을 보존, 소개하는 장점도 있다. 작품 텍스트화는 지금도 현재진행형이며 2011년 3월 15일 현재 등록작품수가 1만권이 넘었다. 고전작가인 모리 오가이, 나츠메 소세키, 아쿠타가와 류노스케, 최근의 작가로는 나카지마 아츠시, 다자이 오사무, 하야시 후미코, 미야모토

In [23]:
texts = []
titles = []
documents_id = []


for i in range(len(wiki_df)):
    wiki_context = wiki_df['text'][i]
    wiki_title = wiki_df['title'][i]
    wiki_docuemnts_id = wiki_df['document_id'][i]

    if wiki_title in wiki_context:
        texts.append(wiki_context)
        titles.append(wiki_title)
        documents_id.append(wiki_docuemnts_id)

wiki_qa_df = pd.DataFrame(data={'text':texts,'title':titles,'document_id':documents_id})
wiki_qa_df.head()

wiki_qa_df.to_csv('wiki_text_title.csv')

In [ ]:
wiki_df[wiki_df['title'] == '아오조라 문고']['text']

763    아오조라 문고(靑空文庫, あおぞらぶんこ|아오조라 분고)는 ‘일본어판 구텐베르크 프로...
764    저자 사망 이후 50년이 지나 저작권이 소멸한 메이지 시대부터 쇼와 시대 초기까지의...
765    아오조라 문고는 자원봉사로 운영되며 열람 역시 무료이다.  서비스 개시 초반에는 보...
766    텍스트 파일을 아오조라 문고에 수록할 때, 텍스트 파일이 갖추어야 할 서식을 '아오...
767    일본어 표기에 많이 쓰이는 올림문자 (후리가나)는 그대로 올려쓰지 않고 '｜'나 '...
Name: text, dtype: object

In [ ]:
wiki_df[wiki_df['title'] == '아오조라 문고']['text'][764]

'저자 사망 이후 50년이 지나 저작권이 소멸한 메이지 시대부터 쇼와 시대 초기까지의 서적 대부분이 존재한다. 외국 번역작품이나 저자가 무료보기를 인정한 현대작품도 포함된다. 장르는 정치부터 취미까지 다양하지만, 비교적 문학작품(시대소설, 추리소설등의 오락작품 포함)이 많다. 유명작가의 작품이 모두 갖춰져있진 않지만 그래도 일본어작품에 관련해서는 충실하게 갖춰진 편이다. (번역작품의 경우 번역저작권을 문제로 수가 많지 않다.)\n\n잘 알려지지 않은 작품을 보존, 소개하는 장점도 있다. 작품 텍스트화는 지금도 현재진행형이며 2011년 3월 15일 현재 등록작품수가 1만권이 넘었다.\n고전작가인 모리 오가이, 나츠메 소세키, 아쿠타가와 류노스케, 최근의 작가로는 나카지마 아츠시, 다자이 오사무, 하야시 후미코, 미야모토 유리코, 호리 다쓰오, 사카구치 안고, 다카무라 고타로, 나가이 가후, 요시카와 에이지 등 인물의 작품이 있다.'

In [ ]:
wiki_df[wiki_df['title'] == 'Safa']['text']

34734    앨범이 발매된 날이 1월 11일인데, 이 날은 시티의 생일이다. 이 앨범은 시티의 ...
Name: text, dtype: object

In [ ]:
texts = []
titles = []

for i in range(len(wiki_df)):
    wiki_context = wiki_df['text'][i]
    wiki_title = wiki_df['title'][i]

    if wiki_title in wiki_context:
        texts.append(wiki_context)
        titles.append(wiki_title)

wiki_qa_df = pd.DataFrame(data={'text':texts,'title':titles})
wiki_qa_df.head()

,text,title
0,매우 다양한 분야의 연구에서 주어진 문제에 대하여 적절한 정보를 수집하고 분석하여 ...,통계학
1,"고대 일본에서 한자의 일부 획을 따서 문자로 삼은 것은 일찍이 7세기부터였으나, ...",가타카나
2,가난 때문에 중학교에 진학하지 못하고 아버지의 농사를 도왔다. 가난에서 벗어나려고 ...,정주영
3,그러나 고 김대중 전 대통령이 제15대 대통령에 당선되어 1998년 2월 25일 국...,정주영
4,선형대수학은 2차원 혹은 3차원의 직교 좌표계에 대한 연구로 부터 시작되었다.\n\...,선형대수학


In [ ]:
qg = pd.read_csv('../question_generation_id.tsv', sep='\t')
qg.columns = ['question', 'answer', 'context_id']

qg = qg.sort_values(by='context_id')

In [ ]:
qg[qg['answer'] == '아오조라 문고']

,question,answer,context_id
20704,일본인터넷 전자도서관의 이름은 무엇인가?,아오조라 문고,3
4743,토미타 아오조라가 자원봉사를 시작한 것은 언제인가?,아오조라 문고,5
18568,음원 파일의 진위여부를 판별할 수 있는 문구는?,아오조라 문고,6
19607,일본 시각장애 독서지원협회는 어떤 문자의 올림문자 사용을 금지하였는가?,아오조라 문고,7
1203,일본인터넷 전자도서관의 이름은 무엇인가?,아오조라 문고,4557
10482,토미타 아오조라가 자원봉사를 시작한 것은 언제인가?,아오조라 문고,4559
2685,음원 파일의 진위여부를 판별할 수 있는 문구는?,아오조라 문고,4560
11261,일본 시각장애 독서지원협회는 어떤 문자의 올림문자 사용을 금지하였는가?,아오조라 문고,4561
